# ase-notebook by example

The classes of `ase-notebook` are:

- :py:class:`~ase_notebook.configuration.ViewConfig`,
  which validates and stores the initialisation configuration.
- :py:class:`~ase_notebook.viewer.AseView`,
  which uses a `ViewConfig` to create a visualisation,
  using one of the backends.

In [ ]:
from ase_notebook import AseView, ViewConfig, get_example_atoms

An `AseView` can be initialised with a `ViewConfig`,
or create the `ViewConfig` *via* keyword arguments.

In [ ]:
config = ViewConfig()
ase_view = AseView(config)

In [ ]:
ase_view = AseView(
    rotations="45x,45y,45z",
    atom_font_size=16,
    axes_length=30,
    canvas_size=(400, 400),
    zoom=1.2,
    show_bonds=True
)
ase_view.config.uc_dash_pattern=(.6,.4)
ase_view.add_miller_plane(
    1, 0, 0, color="green")
ase_view.config

In [ ]:
atoms = get_example_atoms()
atoms

`ase.Atoms` or `pymatgen.Structure`s can be parsed to one of the `AseView` visualisation methods:

- :py:meth:`~ase_notebook.viewer.AseView.make_gui`
  opens a (blocking) GUI window,
  similar to the :py:func:`ase.visualize.view` default.
- :py:meth:`~ase_notebook.viewer.AseView.launch_gui_subprocess`
  opens the GUI in a subprocess,
  allowing multiple windows to be opened at once.
- :py:meth:`~ase_notebook.viewer.AseView.make_svg`
  creates an :py:class:`svgwrite.drawing.Drawing`,
  which will automatically be displayed in the notebook.
- :py:meth:`~ase_notebook.viewer.AseView.make_render`
  creates a :py:class:`~ase_notebook.backend.threejs.RenderContainer`,
  which contains a [pythreejs](pythreejs.readthedocs.io) `Renderer` and [ipywidget](ipywidgets.readthedocs.io) controls,
  that will automatically be displayed in the notebook.

## threejs (3D)

In [ ]:
ase_view.config.canvas_color_background = "blue"
ase_view.config.canvas_background_opacity = 0.2
gui = ase_view.make_render(
    atoms, center_in_uc=True)
gui

.. note::

    The world axes are currently synced to the main scene *via*
    the python kernel, so will only move when it is running.
    Also double-clicking an atom will display information about it.

.. important::

    To use `make_render`, the package must have been installed
    *via* Conda or with the `threejs` extra: `pip install ase-notebook[threejs]`.

Any component of the above GUI can be accessed and displayed separately,
as an additional *view* of the widget.

In [ ]:
gui.element_renderer

To create additional controls,
any element of the scene can be linked to an `ipywidget`.

In [ ]:
from ipywidgets import FloatSlider, jslink
slider = FloatSlider(
    min=0.1, max=10
)
jslink((gui.element_renderer.camera, 'zoom'),
       (slider, 'value'))
slider

If `make_render` is called with `reuse_objects=True`,
then a single atom texture is used for all atoms with the same visual properties (color, opacity, ...).
This widget, for example, will change the color of all Fe atoms above.

In [ ]:
from ipywidgets import ColorPicker, jslink
material = gui.atom_arrays[0]["material_body"]
picker = ColorPicker()
jslink((material, 'color'), (picker, 'value'))
picker

Pythreejs currently can be slow to load,
if the scene contains 1000's of elements
(see [this issue](https://github.com/jupyter-widgets/pythreejs/issues/154)).
To improve this, if `use_atom_arrays=True` then
atoms with the same radius and visual properties,
will be concatenated into a single element.

.. note::

    The atom information picker will not work with this option.

In [ ]:
ase_view.config.atom_show_label = False
ase_view.make_render(
    atoms, center_in_uc=True,
    repeat_uc=(4,4,4), use_atom_arrays=True,
    create_gui=False
)

## SVG (2D)

In [ ]:
ase_view.config.atom_show_label = True
ase_view.config.canvas_background_opacity = 0.0
svg = ase_view.make_svg(atoms, center_in_uc=True)
svg

:py:func:`~ase_notebook.backend.svg.concatenate_svgs`
can also be used to create an of multiple scenes.

.. important::

    To use `concatenate_svgs`, the package must have been installed
    *via* Conda or with the `svgconcat` extra: `pip install ase-notebook[svgconcat]`.

In [ ]:
from ase_notebook import concatenate_svgs

svgs = []
for rot in ["45x,45y,45z", "0x", "90x"]:
    ase_view.config.rotations = rot
    svgs.append(
        ase_view.make_svg(atoms, center_in_uc=True)
    )
concatenate_svgs(
    svgs, max_columns=2, scale=0.5, label=True
)

Finally, if you wish to convert the SVG to a PDF: 

In [ ]:
from ase_notebook.backend.svg import svg_to_pdf
# pdf = svg_to_pdf(svg, "save_file.pdf")